In [16]:
! pip install YT-comments-scrapper==0.3

  Attempting uninstall: YT-comments-scrapper
    Found existing installation: YT-comments-scrapper 0.2
    Uninstalling YT-comments-scrapper-0.2:
      Successfully uninstalled YT-comments-scrapper-0.2


In [72]:
import pandas as pd
import numpy as np
import re
import pickle
from yt_scrapper import YtScrapper
import nltk
from textblob import TextBlob
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [21]:
scrapper = YtScrapper("https://www.youtube.com/watch?v=kuhhT_cBtFU&t=2s",100,"/Users/lindaoranya/downloads/chromedriver")

In [22]:
df = scrapper.scrape_comments()

In [24]:
df.shape

(924, 1)

In [25]:
df_comments = df.copy()

In [81]:
df

,comment
0,LT\nSKIP NAVIGATION\nSIGN IN\n0:02 / 2:48\n#CN...
1,
2,Bodycam video released by the Atlanta Police D...
3,I always wonder why CNN never shows the whole ...
4,Good parenting: “teaches about good and bad pe...
...,...
919,Facts:\n1-The taser was not loaded cause the o...
920,Plis not 'shorten' and 'combine' the video...
921,Guy assaulted the police and it is his fault h...
922,Why won't the Black Lives Matter movement call...


In [82]:
df['polarity'] = df['comment'].apply(lambda x: TextBlob(x).sentiment.polarity)

In [83]:
df['pol_class'] = np.where(df.polarity>0,1,-1)

In [85]:
#get the stopwords for english
stop_words = stopwords.words('english')

In [86]:
#remove no and not from list of stopwords
stop_words.remove('no')
stop_words.remove('not')

In [87]:
stem_obj = PorterStemmer()
lem_obj = WordNetLemmatizer()

In [88]:
corpus = []
#Text Preprocessing without stemming    
for i in range(0,len(df)):
    review = re.sub('[^a-zA-Z]',' ',df.iloc[i,0])
    review = review.lower().split()
    
    #Apply stemming and lemmatization of all words in review
    review_updated = [lem_obj.lemmatize(lem_obj.lemmatize(word,pos='v'),pos='a')\
                     for word in review\
                     if word not in stop_words]
    review_updated = ' '.join(review_updated)
    corpus.append(review_updated)

In [89]:
countVector = CountVectorizer(max_features=2000)
pickle.dump(countVector,open('transform.pkl','wb'))

In [91]:
X = countVector.fit_transform(corpus).toarray()

words = countVector.get_feature_names()
pickle.dump(words,open('vocabulary.pkl','wb'))

In [92]:
y = df.pol_class.values

In [94]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=42) 


In [95]:
#training
rf_classifer = RandomForestClassifier(n_estimators=150,min_samples_split=5)
rf_classifer.fit(X_train,y_train)
rf_classifer.score(X_test,y_test)

0.7482014388489209

In [98]:
rf_classifer.predict(X_test)

-1

In [96]:
pickle.dump(rf_classifer,open('nlp_model.pkl','wb'))